In [1]:
import PIL
import torch
import torchvision.transforms as T
from PIL import Image
import glob
from torch.utils.data import Dataset, DataLoader
from matplotlib import pyplot as plt

In [2]:
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms


Q.1

In [3]:
class Gaussian(object):
    def __init__(self, mean: float, var: float):
        self.mean = mean
        self.var = var
    def __call__(self, img: torch.Tensor) -> torch.Tensor:
        return img + torch.normal(self.mean, self.var, img.size())
preprocess = T.Compose([
    T.ToTensor(),
    T.RandomHorizontalFlip(),
    T.RandomRotation(45),
    Gaussian(0,0.15),
])
class MyDataset(Dataset):
    def __init__(self, transform=None, str="train"):
        self.imgs_path = ".\\data\\cats_and_dogs_filtered\\"+ str + "\\"
        file_list = glob.glob(self.imgs_path + "*")
        self.data = []
        for class_path in file_list:
            class_name = class_path.split("\\")[-1]
            for img_path in glob.glob(class_path + "\\*.jpg"):
                self.data.append([img_path, class_name])
        self.class_map = {"dogs" : 0, "cats": 1}
        self.transform = transform
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        img_path, class_name = self.data[idx]
        img = PIL.Image.open(img_path)
        class_id = self.class_map[class_name]
        class_id = torch.tensor(class_id)
        if self.transform:
            img = self.transform(img)
        return img, class_id
dataset = MyDataset(transform=preprocess,str = "train")
dataloader = DataLoader(dataset)

In [4]:
#The following code displays augmented images
i=0
for data in iter(dataloader):
    img,_ = data
    tI = T.ToPILImage()
    img = tI(img.squeeze())
    plt.imshow(img)
    plt.show()
    i = i + 1
    if i == 3:
        break
#Extend the code to include training loop

In [5]:
# Define your neural network architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, 1)
        self.conv2 = nn.Conv2d(16, 32, 3, 1)
        self.fc1 = nn.Linear(32*14*14, 128)
        self.fc2 = nn.Linear(128, 2)  # 2 classes (cat and dog)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 32*14*14)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Load the model, loss function, and optimizer
model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=0.01)  # L2 regularization

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(dataloader):
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print(f'Epoch {epoch+1}, Batch {i}, Loss: {loss.item()}')

# Save the trained model
torch.save(model.state_dict(), 'model.pth')

In [6]:

# Define your neural network architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, 1)
        self.conv2 = nn.Conv2d(16, 32, 3, 1)
        self.fc1 = nn.Linear(32*14*14, 128)
        self.fc2 = nn.Linear(128, 2)  # 2 classes (cat and dog)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 32*14*14)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

class MyDataset(Dataset):
    def __init__(self, transform=None, str="train"):
        self.imgs_path = ".\\data\\cats_and_dogs_filtered\\" + str + "\\"
        file_list = glob.glob(self.imgs_path + "*")
        self.data = []
        for class_path in file_list:
            class_name = class_path.split("\\")[-1]
            for img_path in glob.glob(class_path + "\\*.jpg"):
                self.data.append([img_path, class_name])
        self.class_map = {"dogs": 0, "cats": 1}
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, class_name = self.data[idx]
        img = PIL.Image.open(img_path)
        class_id = self.class_map[class_name]
        class_id = torch.tensor(class_id)
        if self.transform:
            img = self.transform(img)
        return img, class_id

# Load the dataset
transform = T.Compose([
    T.Resize((64, 64)),
    T.ToTensor(),
])

train_dataset = MyDataset(transform=transform, str="train")
test_dataset = MyDataset(transform=transform, str="test")
train_loader = DataLoader(train_dataset)
test_loader = DataLoader(test_dataset)

# Define the model, loss function, and optimizer
model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=0.01)  # L2 regularization

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
    # Validation
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            accuracy = correct / total
            print(f'Epoch {epoch+1}, Accuracy: {accuracy}')

# Observe the impact of regularization on the weight parameters
for name, param in model.named_parameters():
    if 'weight' in name:
        l2_norm = torch.norm(param, p=2)  # L2 norm of weights
        print(f'Layer: {name}, L2 Norm: {l2_norm.item()}')


Layer: conv1.weight, L2 Norm: 2.3536782264709473
Layer: conv2.weight, L2 Norm: 3.2647111415863037
Layer: fc1.weight, L2 Norm: 6.531321048736572
Layer: fc2.weight, L2 Norm: 0.797450602054596


Q.2a. L1 regularization using optimizer's weight decay:

In [7]:

# Define your neural network architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, 1)
        self.conv2 = nn.Conv2d(16, 32, 3, 1)
        self.fc1 = nn.Linear(32*14*14, 128)
        self.fc2 = nn.Linear(128, 2)  # 2 classes (cat and dog)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 32*14*14)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x



# Define the model, loss function, and optimizer with L1 regularization
model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=0.01)  # L1 regularization

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
    # Validation
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            accuracy = correct / total
            print(f'Epoch {epoch+1}, Accuracy: {accuracy}')

# Observe the impact of L1 regularization on the weight parameters
for name, param in model.named_parameters():
    if 'weight' in name:
        l1_norm = torch.norm(param, p=1)  # L1 norm of weights
        print(f'Layer: {name}, L1 Norm: {l1_norm.item()}')


Layer: conv1.weight, L1 Norm: 40.86850357055664
Layer: conv2.weight, L1 Norm: 195.38363647460938
Layer: fc1.weight, L1 Norm: 5066.7001953125
Layer: fc2.weight, L1 Norm: 11.316882133483887


In [8]:

# Initialize the L1 norm to zero
l1_norm = 0

# Calculate the L1 norm of the weights
for param in model.parameters():
    if 'weight' in name:
        l1_norm += torch.sum(torch.abs(param))

print(f'Total L1 Norm: {l1_norm}')




Total L1 Norm: 0
